In [1]:
import numpy as np
import scipy as sp

import scipy
from scipy.stats import multivariate_normal
from scipy       import optimize

import skimage

import matplotlib.pyplot as plt

import pylab

from glob import glob
import sys
import os

import itk
from itkwidgets import view

from tqdm import tqdm

import math

from glob import glob

import matplotlib.pyplot as plt

import itk

import av
import ffmpeg

import json

import site
site.addsitedir('../../ARGUS')
from ARGUSUtils_IO import *
from ARGUSUtils_Linearization import *

In [8]:
def convert_boxes_to_linear(pathname):
    data_paths = sorted(glob(pathname+"-Annotations/*f?????.mha"))
    for i in range(len(data_paths)):
        bxfile = pathname+"-Annotations/"+os.path.basename(data_paths[i])[:-4]+".boxes.json"
        print(bxfile)

        itkimg = itk.imread(data_paths[i], itk.UC)
        depth,zoom,offsetX,offsetY = get_depth_and_zoom_C52(itkimg)

        itkboximg_linear = linearize_image(itkimg,depth,zoom,offsetY)
        itkboximg_linear.FillBuffer(0)
        itkimg.FillBuffer(0)
        boximg = itk.GetArrayViewFromImage(itkimg)

        with open(bxfile) as f:
          data = json.load(f)
        slices = data['boxes']
        num_box_slices = len(slices)
        box_count = 0
        for s in range(num_box_slices):
            boxes = slices[s]['boxes']
            num_boxes = len(boxes)
            for b in range(num_boxes):
                box_indx = boxes[b]['indices']
                print("   ",box_indx)
                start = [int(item) for item in box_indx[0]]
                end = [int(item) for item in box_indx[1]]
                if(start[0]>end[0]):
                    start[0], end[0] = end[0], start[0]
                if(start[1]>end[1]):
                    start[1], end[1] = end[1], start[1]
                boximg[start[1]:end[1],start[0]:end[0]] = box_count+1
                box_count += 1
        itkboximg_linear = linearize_image(itkimg,depth,zoom,offsetY,False)  #use False to not interpolate
        bxfile = pathname+"-Annotations-Linear/"+os.path.basename(data_paths[i])[:-4]+".boxes.mha"
        itk.imwrite(itkboximg_linear, bxfile)
        labelmap = itk.GetArrayFromImage(itkboximg_linear).astype(int)
        props = skimage.measure.regionprops(labelmap)
        if box_count != len(props):
            print("ERROR: # of linearized boxes (",len(props),") != number of boxes (",box_count,")")
        box_count = 0
        for s in range(num_box_slices):
            boxes = slices[s]['boxes']
            num_boxes = len(boxes)
            for b in range(num_boxes):
                bbox = props[box_count].bbox
                boxes[b]['indices'] = [[bbox[1],bbox[0],0],[bbox[3],bbox[2],0]]
                box_count += 1
        bxfile = pathname+"-Annotations-Linear/"+os.path.basename(data_paths[i])[:-4]+".boxes.json"
        with open(bxfile, 'w') as outfile:
            json.dump(data, outfile)


In [12]:
pathname = "../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding"
convert_boxes_to_linear(pathname)

../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/Image_262499828648_clean.mp4-00000-f00000.boxes.json
    [[828.75, 450.0, 0.0], [911.25, 498.75, 0.0]]
    [[976.875, 468.75, 0.0], [1044.375, 504.375, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/Image_262499828648_clean.mp4-00001-f00004.boxes.json
    [[971.25, 465.0, 0.0], [1042.5, 504.375, 0.0]]
    [[836.25, 453.75, 0.0], [911.25, 496.875, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/Image_262499828648_clean.mp4-00002-f00006.boxes.json
    [[830.625, 457.5, 0.0], [901.875, 496.875, 0.0]]
    [[967.5, 470.625, 0.0], [1068.75, 513.75, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/Image_262499828648_clean.mp4-00003-f00011.boxes.json
    [[976.875, 474.375, 0.0], [1055.625, 506.25, 0.0]]
    [[838.125, 463.125, 0.0], [913.125, 508.125, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/Image_262499828648_clean.mp4-00004-f00017.boxes.json
  

../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_1394469579519_clean.mp4-00008-f00055.boxes.json
    [[975.0, 393.75, 0.0], [1076.25, 442.5, 0.0]]
    [[804.375, 401.25, 0.0], [877.5, 455.625, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_1394469579519_clean.mp4-00009-f00060.boxes.json
    [[965.625, 406.875, 0.0], [1070.625, 448.125, 0.0]]
    [[808.125, 399.375, 0.0], [883.125, 450.0, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_1404802450036_clean.mp4-00000-f00000.boxes.json
    [[995.625, 401.25, 0.0], [1080.0, 461.25, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_1404802450036_clean.mp4-00001-f00010.boxes.json
    [[988.125, 412.5, 0.0], [1091.25, 461.25, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_1404802450036_clean.mp4-00002-f00017.boxes.json
    [[990.0, 405.0, 0.0], [1102.5, 451.875, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annot

../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_1884162273498_clean.mov-00009-f00060.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[828.75, 324.375, 0.0], [954.375, 412.5, 0.0]]
    [[993.75, 365.625, 0.0], [1102.5, 431.25, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_1895283541879_clean.mov-00000-f00000.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[663.75, 311.25, 0.0], [772.5, 401.25, 0.0]]
    [[825.0, 335.625, 0.0], [950.625, 418.125, 0.0]]
    [[997.5, 360.0, 0.0], [1098.75, 455.625, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_1895283541879_clean.mov-00001-f00007.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[660.0, 301.875, 0.0], [766.875, 375.0, 0.0]]
    [[815.625, 341.25, 0.0], [937.5, 403.125, 0.0]]
    [[999.375, 376.875, 0.0], [1111.875, 463.125, 0.0]]
   Resampl

    [[988.125, 459.375, 0.0], [1066.875, 496.875, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_267456908021_clean.mp4-00006-f00039.boxes.json
    [[845.625, 459.375, 0.0], [931.875, 506.25, 0.0]]
    [[995.625, 465.0, 0.0], [1057.5, 502.5, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_267456908021_clean.mp4-00007-f00040.boxes.json
    [[849.375, 457.5, 0.0], [933.75, 500.625, 0.0]]
    [[986.25, 461.25, 0.0], [1065.0, 498.75, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_267456908021_clean.mp4-00008-f00049.boxes.json
    [[858.75, 468.75, 0.0], [941.25, 515.625, 0.0]]
    [[993.75, 468.75, 0.0], [1078.125, 508.125, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_267456908021_clean.mp4-00009-f00060.boxes.json
    [[686.25, 480.0, 0.0], [761.25, 517.5, 0.0]]
    [[853.125, 468.75, 0.0], [937.5, 511.875, 0.0]]
    [[1001.25, 463.125, 0.0], [1083.75, 498.75, 0.0]]
../../Data/Train

../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_2743083265515_CLEAN.mov-00006-f00028.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[975.0, 474.375, 0.0], [1080.0, 523.125, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_2743083265515_CLEAN.mov-00007-f00032.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[984.375, 466.875, 0.0], [1095.0, 517.5, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_2743083265515_CLEAN.mov-00008-f00034.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[787.5, 506.25, 0.0], [901.875, 562.5, 0.0]]
    [[982.5, 465.0, 0.0], [1096.875, 515.625, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_2743083265515_CLEAN.mov-00009-f00040.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[965.625, 

    [[915.0, 292.5, 0.0], [1066.875, 339.375, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_584357289931_clean.mov-00009-f00040.boxes.json
    [[913.125, 273.75, 0.0], [1057.5, 339.375, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_588695055398_clean.mov-00001-f00005.boxes.json
    [[924.375, 275.625, 0.0], [1066.875, 358.125, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_588695055398_clean.mov-00002-f00007.boxes.json
    [[900.0, 285.0, 0.0], [1087.5, 341.25, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_588695055398_clean.mov-00003-f00008.boxes.json
    [[911.25, 281.25, 0.0], [1087.5, 350.625, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_588695055398_clean.mov-00004-f00010.boxes.json
    [[916.875, 281.25, 0.0], [1070.625, 350.625, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_588695055398_clean.mov-00005-f00017.boxe

../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_610066411380_CLEAN.mov-00006-f00020.boxes.json
   Resampling with zoom = 0.8048048048048048
    [[928.125, 414.375, 0.0], [1018.125, 476.25, 0.0]]
   Resampling with zoom = 0.8048048048048048
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_610066411380_CLEAN.mov-00007-f00037.boxes.json
   Resampling with zoom = 0.8048048048048048
    [[901.875, 420.0, 0.0], [1014.375, 487.5, 0.0]]
   Resampling with zoom = 0.8048048048048048
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_610066411380_CLEAN.mov-00008-f00040.boxes.json
   Resampling with zoom = 0.8048048048048048
    [[907.5, 397.5, 0.0], [1005.0, 481.875, 0.0]]
   Resampling with zoom = 0.8048048048048048
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_614587120545_clean.mov-00000-f00000.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[840.0, 260.625, 0.0], [1029.375, 343.125, 0.0]]
   Resampling 

../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_6370410622099_CLEAN.mov-00004-f00020.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[798.75, 436.875, 0.0], [877.5, 491.25, 0.0]]
    [[1018.125, 448.125, 0.0], [1098.75, 504.375, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_6370410622099_CLEAN.mov-00005-f00035.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[993.75, 427.5, 0.0], [1115.625, 498.75, 0.0]]
    [[793.125, 427.5, 0.0], [894.375, 504.375, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/image_6370410622099_CLEAN.mov-00006-f00040.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[804.375, 442.5, 0.0], [875.625, 504.375, 0.0]]
    [[1014.375, 433.125, 0.0], [1102.5, 481.875, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations/

In [17]:
pathname = "../../Data/TrainingData/AR-UNet/BAMC-PTXSliding"
convert_boxes_to_linear(pathname)

../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_10391571128899_CLEAN.mp4-00000-f00000.boxes.json
    [[913.125, 369.375, 0.0], [990.0, 414.375, 0.0]]
    [[1042.5, 307.5, 0.0], [1115.625, 375.0, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_10391571128899_CLEAN.mp4-00001-f00007.boxes.json
    [[1035.0, 303.75, 0.0], [1143.75, 363.75, 0.0]]
    [[926.25, 361.875, 0.0], [991.875, 406.875, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_10391571128899_CLEAN.mp4-00002-f00020.boxes.json
    [[922.5, 371.25, 0.0], [993.75, 406.875, 0.0]]
    [[1044.375, 313.125, 0.0], [1136.25, 361.875, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_10391571128899_CLEAN.mp4-00003-f00032.boxes.json
    [[922.5, 380.625, 0.0], [999.375, 410.625, 0.0]]
    [[1059.375, 318.75, 0.0], [1155.0, 352.5, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_10391571128899_CLEAN.mp4-00004-f00038.boxes.json
   

../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_104548309385533_CLEAN.mov-00005-f00020.boxes.json
   Resampling with zoom = 0.8048048048048048
    [[924.375, 433.125, 0.0], [1050.0, 511.875, 0.0]]
   Resampling with zoom = 0.8048048048048048
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_104548309385533_CLEAN.mov-00006-f00037.boxes.json
   Resampling with zoom = 0.8048048048048048
    [[924.375, 433.125, 0.0], [1046.25, 506.25, 0.0]]
   Resampling with zoom = 0.8048048048048048
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_104548309385533_CLEAN.mov-00007-f00040.boxes.json
   Resampling with zoom = 0.8048048048048048
    [[918.75, 433.125, 0.0], [1038.75, 508.125, 0.0]]
   Resampling with zoom = 0.8048048048048048
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_104548309385533_CLEAN.mov-00008-f00054.boxes.json
   Resampling with zoom = 0.8048048048048048
    [[911.25, 421.875, 0.0], [1048.125, 521.25, 0.0]]
   Re

../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_1139765223418_CLEAN.mov-00005-f00034.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[834.375, 315.0, 0.0], [920.625, 380.625, 0.0]]
    [[1050.0, 320.625, 0.0], [1162.5, 382.5, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_1139765223418_CLEAN.mov-00006-f00037.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[845.625, 315.0, 0.0], [920.625, 375.0, 0.0]]
    [[1042.5, 320.625, 0.0], [1153.125, 375.0, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_1139765223418_CLEAN.mov-00007-f00041.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[832.5, 328.125, 0.0], [930.0, 391.875, 0.0]]
    [[1038.75, 315.0, 0.0], [1147.5, 376.875, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_11397652234

../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_128688523296793_CLEAN.mov-00002-f00011.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[1008.75, 382.5, 0.0], [1173.75, 429.375, 0.0]]
    [[727.5, 358.125, 0.0], [866.25, 451.875, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_128688523296793_CLEAN.mov-00003-f00014.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[1001.25, 363.75, 0.0], [1183.125, 450.0, 0.0]]
    [[723.75, 360.0, 0.0], [862.5, 431.25, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_128688523296793_CLEAN.mov-00004-f00017.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[1008.75, 369.375, 0.0], [1171.875, 450.0, 0.0]]
    [[706.875, 360.0, 0.0], [871.875, 423.75, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_128

../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_1499268364374_clean.mov-00004-f00017.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[725.625, 341.25, 0.0], [793.125, 397.5, 0.0]]
    [[886.875, 363.75, 0.0], [969.375, 418.125, 0.0]]
    [[1044.375, 365.625, 0.0], [1145.625, 431.25, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_1499268364374_clean.mov-00005-f00020.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[881.25, 346.875, 0.0], [960.0, 405.0, 0.0]]
    [[1057.5, 376.875, 0.0], [1125.0, 438.75, 0.0]]
    [[723.75, 326.25, 0.0], [810.0, 408.75, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_1499268364374_clean.mov-00006-f00045.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[868.125, 341.25, 0.0], [975.0, 410.625, 0.0]]
    [[1042.5, 375.0, 0.0], [1143.75, 436.875, 0.0]]
    [[723.75, 3

    [[780.0, 343.125, 0.0], [856.875, 420.0, 0.0]]
    [[978.75, 373.125, 0.0], [1040.625, 433.125, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_1901852337971_clean.mp4-00004-f00015.boxes.json
    [[770.625, 326.25, 0.0], [871.875, 405.0, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_1901852337971_clean.mp4-00005-f00017.boxes.json
    [[765.0, 326.25, 0.0], [868.125, 401.25, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_1901852337971_clean.mp4-00006-f00020.boxes.json
    [[759.375, 343.125, 0.0], [862.5, 405.0, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_1901852337971_clean.mp4-00007-f00050.boxes.json
    [[963.75, 369.375, 0.0], [1025.625, 425.625, 0.0]]
    [[763.125, 346.875, 0.0], [862.5, 408.75, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_1901852337971_clean.mp4-00008-f00059.boxes.json
    [[772.5, 343.125, 0.0], [853.125, 391.875, 0.0]]
    [[965.

    [[918.75, 543.75, 0.0], [1115.625, 654.375, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_3315947589826_clean.mov-00009-f00056.boxes.json
    [[933.75, 556.875, 0.0], [1130.625, 646.875, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_3320344386805_clean.mp4-00000-f00000.boxes.json
    [[855.0, 390.0, 0.0], [1021.875, 451.875, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_3320344386805_clean.mp4-00001-f00013.boxes.json
    [[1021.875, 352.5, 0.0], [856.875, 470.625, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_3320344386805_clean.mp4-00002-f00020.boxes.json
    [[1018.125, 356.25, 0.0], [875.625, 455.625, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_3320344386805_clean.mp4-00003-f00038.boxes.json
    [[849.375, 363.75, 0.0], [1020.0, 457.5, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_3320344386805_clean.mp4-00004-f00040.boxes.

../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_3384882513134_clean.mp4-00004-f00020.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[780.0, 420.0, 0.0], [881.25, 474.375, 0.0]]
    [[1012.5, 461.25, 0.0], [1138.125, 510.0, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_3384882513134_clean.mp4-00005-f00027.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[791.25, 406.875, 0.0], [886.875, 481.875, 0.0]]
    [[980.625, 446.25, 0.0], [1078.125, 517.5, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_3384882513134_clean.mp4-00006-f00029.boxes.json
   Resampling with zoom = 1.2627627627627627
    [[787.5, 406.875, 0.0], [875.625, 487.5, 0.0]]
    [[986.25, 442.5, 0.0], [1093.125, 513.75, 0.0]]
   Resampling with zoom = 1.2627627627627627
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_3384882513

    [[836.25, 333.75, 0.0], [958.125, 386.25, 0.0]]
    [[1125.0, 253.125, 0.0], [1010.625, 335.625, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_73815992352100_clean.mp4-00002-f00012.boxes.json
    [[841.875, 341.25, 0.0], [946.875, 390.0, 0.0]]
    [[1016.25, 290.625, 0.0], [1130.625, 343.125, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_73815992352100_clean.mp4-00003-f00018.boxes.json
    [[1021.875, 296.25, 0.0], [1132.5, 328.125, 0.0]]
    [[851.25, 337.5, 0.0], [961.875, 380.625, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_73815992352100_clean.mp4-00004-f00020.boxes.json
    [[849.375, 343.125, 0.0], [954.375, 382.5, 0.0]]
    [[1014.375, 277.5, 0.0], [1134.375, 341.25, 0.0]]
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations/image_73815992352100_clean.mp4-00005-f00029.boxes.json
    [[1006.875, 281.25, 0.0], [1117.5, 339.375, 0.0]]
    [[856.875, 333.75, 0.0], [948.75, 380.625, 0.0]]
../..